# Data Scraping
This is a data scraper used to extract the advanced NBA team statistics from [stats.nba.com](https://stats.nba.com/). 6 different advanced stat categories were used (drives, catch and shoot, passing, pullup shooting, touches, and hustle). Each of these stats were combined into one single advanced stats dataframe where the rows were each team and 
the columns were the statistics. 

In [12]:
from selenium import webdriver
from bs4 import BeautifulSoup as soup 
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np

In [9]:
## returns dataframe of the given statistic with the NBA team as the index
def scrape(stat,year):
    d = webdriver.Chrome(ChromeDriverManager().install())
    d.get('https://stats.nba.com/teams/'+stat+'/?Season='+year+'&SeasonType=Regular%20Season')
    s = soup(d.page_source, 'html.parser').find('table', {'class':'table'})
    headers, [_, *data] = [i.text for i in s.find_all('th')], [[i.text for i in b.find_all('td')] for b in s.find_all('tr')]
    final_data = [i for i in data if len(i) > 1]
    data_attrs = [dict(zip(headers, i)) for i in final_data]
    df = pd.DataFrame(data_attrs)
    df['Team'] = df['Team'].str.replace(r'\n', '')
    df = df.set_index("Team")
    return df

In [10]:
## creates entire data frame with all 6 tracking statistics given the season (XXXX-XX)
def create_trackingData(year):
    drives = scrape('drives', year)
    catch_shoot = scrape('catch-shoot', year)
    passing = scrape('passing', year)
    pullup = scrape('pullup', year)
    touches = scrape('touches', year)
    drives = drives[["DRIVES"]]
    catch_shoot = catch_shoot[["PTS"]]
    passing = passing[["PassesMade", "SecondaryAST"]]
    touches = touches[["TOUCHES", "PaintTouches", "PostUps"]]
    pullup = pd.DataFrame(pullup.iloc[:,15])
    pullup.columns = ["PullUpPoints"]
    
    hustle = pd.read_excel("data/hustle"+year+".xlsx")
    misc = pd.read_excel("data/misc"+year+".xlsx")
    misc = misc.set_index("Team")
    hustle = hustle.rename(columns = {'TEAM' : 'Team'})
    hustle = hustle.set_index("Team")
    full_advanced = pd.concat([drives, catch_shoot, pullup,touches, passing, hustle, misc], axis=1, sort = True)
    full_advanced.to_excel("tracking"+year+".xlsx")

In [72]:
create_trackingData('2018-19')

Most of the data will consist of team statistics from [basketball-reference.com](https://www.basketball-reference.com/). Three tables from the team statistics page from each season will be used. They can easily be downloaded as csv or excel files. Then the tables will be cleaned individually and combined together so that the indices are all the NBA teams and the columns are the statistical categories with distinct names. As of now these tables will be manually downloaded from basketball-reference.

In [41]:
## 2019-20 : Unfinished Season
basic20 = pd.read_csv("data/basic2019-20.csv")

In [53]:
## 2018-19 Finished Season
basic19 = pd.read_csv("data/basic2018-19.csv")
basic_opp19 = pd.read_csv("data/basic_opp2018-19.csv")
advanced19 = pd.read_excel("data/advanced2018-19.xlsx").set_index("Team")

In [13]:
## 2017-18 Finished Season
basic18 = pd.read_csv("data/basic2017-18.csv")
basic_opp18 = pd.read_csv("data/basic_opp2017-18.csv")
advanced18 = pd.read_excel("data/advanced2017-18.xlsx").set_index("Team")

In [63]:
## 2016-17 Finished Season
basic17 = pd.read_csv("data/basic2016-17.csv")
basic_opp17 = pd.read_csv("data/basic_opp2016-17.csv")
advanced17 = pd.read_excel("data/advanced2016-17.xlsx").set_index("Team")

In [117]:
## 2015-16 Finished Season
basic16 = pd.read_csv("data/basic2015-16.csv")
basic_opp16 = pd.read_csv("data/basic_opp2015-16.csv")
advanced16 = pd.read_excel("data/advanced2015-16.xlsx").set_index("Team")

# Data Cleaning
Once all three tables (team, opponents, advanced) are merged into a single dataframe they will be cleaned. Games played and minutes will be dropped because they are the same for all teams. Then an extra column will be added to introduce 'Star Power' as a metric. Despite team success during the regular season, having a star player provides somewhat of an intangible boost as these players are capable of special things during the playoffs. As a result teams with a player in the top 5 of the MVP voting receive a 1. The rest of teams will have 0. Statistics, which are redundant as other statistics capture essentially the same thing, will be dropped. The seeding of each team will be manually assigned and adding a column called 'Seed'. Finally the playoff results of each team for each season will be manually assigned, essentially 'labeling' each of team data points, by adding a column called 'Playoffs'.

In [15]:
def clean(df, opp):
    if (opp):
        df.drop(["Rk","G", "MP"], axis = 1, inplace = True)
        df.columns = ['op_' + str(col) for col in df.columns]
        df = df.rename(columns = {'op_Team' : 'Team'})
        df['Team'] = df['Team'].str.replace(r'*', '')
        df = df.set_index("Team")
    else:  
        df['Team'] = df['Team'].str.replace(r'*', '')
        df.drop(["Rk","G", "MP"], axis = 1, inplace = True)
        df = df.set_index("Team")
    return df

### 2018-19 Season Data

In [47]:
basic20 = clean(basic20, False)
basic19 = clean(basic19, False)
basic_opp19 = clean(basic_opp19, True)

In [48]:
basic19 = pd.concat([basic19, basic_opp19], axis=1, sort = True)
basic19 = basic19.round(3)
basic19.insert(len(basic19.columns),'Star_Power',0)
basic19.loc["Milwaukee Bucks", "Star_Power"] = 1
basic19.loc["Houston Rockets", "Star_Power"] = 1
basic19.loc["Oklahoma City Thunder", "Star_Power"] = 1
basic19.loc["Denver Nuggets", "Star_Power"] = 1
basic19.loc["Golden State Warriors", "Star_Power"] = 1

In [57]:
s2019 = pd.read_excel("data/basic2018-19_labels.xlsx")
s2019 = s2019.set_index("Team")
s2019 = pd.concat([s2019, advanced19], axis=1, sort = True)
full_advanced2019 = pd.read_excel("data/full_advanced2018-19.xlsx").set_index("Team")
s2019_full = pd.concat([s2019, advanced19, full_advanced2019], axis=1, sort = True)
s2019.drop(["op_STL", "op_BLK", "op_PF", "op_FT%", "op_FTA", "op_DRB", "op_TRB"], axis = 1, inplace = True)
# s2019.to_excel("data/s2019.xlsx")

### 2017-18 Season Data

In [16]:
basic18 = clean(basic18, False)
basic_opp18 = clean(basic_opp18, True)
basic18 = pd.concat([basic18, basic_opp18], axis=1, sort = True)
basic18 = basic18.round(3)

In [17]:
basic18.insert(len(basic18.columns),'Star_Power',0)
basic18.loc["Cleveland Cavaliers", "Star_Power"] = 1
basic18.loc["Houston Rockets", "Star_Power"] = 1
basic18.loc["Oklahoma City Thunder", "Star_Power"] = 1
basic18.loc["Portland Trail Blazers", "Star_Power"] = 1
basic18.loc["New Orleans Pelicans", "Star_Power"] = 1

In [18]:
s2018 = pd.read_excel("data/basic2017-18_labels.xlsx")
s2018 = s2018.set_index("Team")
s2018 = pd.concat([s2018, advanced18], axis=1, sort = True)
s2018.drop(["op_STL", "op_BLK", "op_PF", "op_FT%", "op_FTA", "op_DRB", "op_TRB"], axis = 1, inplace = True)
# s2018.to_excel("data/s2018.xlsx")
s2018.to_csv("data/s2018.csv")

### 2016-17 Season Data

In [64]:
basic17 = clean(basic17, False)
basic_opp17 = clean(basic_opp17, True)
basic17 = pd.concat([basic17, basic_opp17], axis=1, sort = True)
basic17 = basic17.round(3)

In [65]:
basic17 = pd.concat([basic17, basic_opp17], axis=1, sort = True)
basic17 = basic17.round(3)
basic17.insert(len(basic17.columns),'Star_Power',0)
basic17.loc["Cleveland Cavaliers", "Star_Power"] = 1
basic17.loc["Houston Rockets", "Star_Power"] = 1
basic17.loc["Oklahoma City Thunder", "Star_Power"] = 1
basic17.loc["Boston Celtics", "Star_Power"] = 1
basic17.loc["San Antonio Spurs", "Star_Power"] = 1

In [66]:
s2017 = pd.read_excel("data/basic2016-17_labels.xlsx")
s2017 = s2017.set_index("Team")
s2017 = pd.concat([s2017, advanced17], axis=1, sort = True)
s2017.drop(["op_STL", "op_BLK", "op_PF", "op_FT%", "op_FTA", "op_DRB", "op_TRB"], axis = 1, inplace = True)
# s2017.to_excel("data/s2017.xlsx")

### 2015-16 Season Data

In [118]:
basic16 = clean(basic16, False)
basic_opp16 = clean(basic_opp16, True)
basic16 = pd.concat([basic16, basic_opp16], axis=1, sort = True)
basic16 = basic16.round(3)

In [119]:
basic16 = pd.concat([basic16, basic_opp16], axis=1, sort = True)
basic16 = basic16.round(3)
basic16.insert(len(basic16.columns),'Star_Power',0)
basic16.loc["Cleveland Cavaliers", "Star_Power"] = 1
basic16.loc["Los Angeles Clippers", "Star_Power"] = 1
basic16.loc["Oklahoma City Thunder", "Star_Power"] = 1
basic16.loc["Golden State Warriors", "Star_Power"] = 1
basic16.loc["San Antonio Spurs", "Star_Power"] = 1

In [5]:
s2016 = pd.read_excel("data/basic2015-16_labels.xlsx")
s2016 = s2016.set_index("Team")
s2016 = pd.concat([s2016, advanced16], axis=1, sort = True)
s2016.drop(["op_STL", "op_BLK", "op_PF", "op_FT%", "op_FTA", "op_DRB", "op_TRB"], axis = 1, inplace = True)
s2016.to_excel("data/s2015-16.xlsx")

### 2014-15 Season Data

In [1]:
basic15 = pd.read_csv("data/basic2014-15.csv")
basic_opp15 = pd.read_csv("data/basic_opp2014-15.csv")
advanced15 = pd.read_excel("data/advanced2014-15.xlsx").set_index("Team")

In [2]:
basic15 = clean(basic15, False)
basic_opp15 = clean(basic_opp15, True)
basic15 = pd.concat([basic15, basic_opp15], axis=1, sort = True)
basic15 = basic15.round(3)

In [3]:
basic15.insert(len(basic15.columns),'Star_Power',0)
basic15.loc["Cleveland Cavaliers", "Star_Power"] = 1
basic15.loc["New Orleans Pelicans", "Star_Power"] = 1
basic15.loc["Oklahoma City Thunder", "Star_Power"] = 1
basic15.loc["Golden State Warriors", "Star_Power"] = 1
basic15.loc["Houston Rockets", "Star_Power"] = 1

In [4]:
s2015 = pd.read_excel("data/basic2014-15_labels.xlsx")
s2015 = s2015.set_index("Team")
s2015 = pd.concat([s2015, advanced15], axis=1, sort = True)
s2015.drop(["op_STL", "op_BLK", "op_PF", "op_FT%", "op_FTA", "op_DRB", "op_TRB"], axis = 1, inplace = True)
s2015.to_excel("data/s2014-15.xlsx")

### 2013-14 Season Data

In [6]:
basic14 = pd.read_csv("data/basic2013-14.csv")
basic_opp14 = pd.read_csv("data/basic_opp2013-14.csv")
advanced14 = pd.read_excel("data/advanced2013-14.xlsx").set_index("Team")

In [7]:
basic14 = clean(basic14, False)
basic_opp14 = clean(basic_opp14, True)
basic14 = pd.concat([basic14, basic_opp14], axis=1, sort = True)
basic14 = basic14.round(3)

In [ ]:
basic14.insert(len(basic14.columns),'Star_Power',0)
basic14.loc["Miami Heat", "Star_Power"] = 1
basic14.loc["Los Angeles Clippers", "Star_Power"] = 1
basic14.loc["Oklahoma City Thunder", "Star_Power"] = 1
basic14.loc["Chicago Bulls", "Star_Power"] = 1
basic14.loc["Houston Rockets", "Star_Power"] = 1

In [8]:
s2014 = pd.read_excel("data/basic2013-14_labels.xlsx")
s2014 = s2014.set_index("Team")
s2014 = pd.concat([s2014, advanced14], axis=1, sort = True)
s2014.drop(["op_STL", "op_BLK", "op_PF", "op_FT%", "op_FTA", "op_DRB", "op_TRB"], axis = 1, inplace = True)
s2014.to_excel("data/s2013-14.xlsx")